On the next stage I need to download file with ortologus rows in the system. I have mounted google drive with project files (in the future I'm going to use GitHub for these purposes). 

In [1]:
import os
import sys
import argparse
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument("organism_name", type=str)

arguments = parser.parse_args()

organism_name = arguments.organism_name
lst = organism_name.split()
folder_name = f"{lst[0][0]}_{lst[1]}"

### Orto rows with singletons, but without paralog-containing rows

In [12]:
# Uploading and modificating orto rows tables
orto_rows = pd.read_csv(f"../{folder_name}/data/{folder_name}.proteinortho.tsv", sep="\t") # Uploading dataframe with orto rows
orto_rows = orto_rows.rename(columns = {"# Species": "Strains"})
orto_rows["ortologus_row"] = orto_rows.index + 1 # Creating ortorows numbers

# Uploading first csv-table and creating a new column in it
orto_rows_list = orto_rows.index
df1 = pd.read_csv(f"../{folder_name}/data/First_table.csv")
df1["ortologus_row"] = 0

#Filling orto_row column (sounds like an oxymoron)
organism = organism_name
for strain in tqdm(set(df1["p_c_unity"])):
    maskstring = f"{folder_name}{strain}.fasta"   
    orrow = orto_rows.loc[:, maskstring].str[:14]
    for index in orto_rows_list:
        df1.loc[df1[df1["id"] == orrow[index]].index, "ortologus_row"] = index + 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:55<00:00,  2.04s/it]


In [3]:
# Number of the paralogs
print("Number of the paralogs =", sum(orto_rows.query("Genes > Strains").Genes) - sum(orto_rows.query("Genes > Strains").Strains))

Number of the paralogs = 65


In [13]:
# Creating a subset without pararows
pararows_numbers = orto_rows.query("Genes > Strains").ortologus_row
df1 = df1.query("ortologus_row not in @pararows_numbers").query("ortologus_row != 0")

In [14]:
# Compiling data about start-codons of ortologus rows
start_codon_per_row = df1.groupby("ortologus_row", as_index=False).agg({"start_codone": ".".join})
start_codon_per_row["start_codone"]
orr_start_list = []
for number in tqdm(range(len(start_codon_per_row))):
    orr_start_list.append(start_codon_per_row.iloc[number, 1].split("."))
start_codons = pd.DataFrame(
    {
        "ortologus_row": start_codon_per_row["ortologus_row"],
        "start_codons": pd.Series(orr_start_list),
        "ATG": 0.0,
        "GTG": 0.0,
        "TTG": 0.0,
    }
)

# Computing frequencies of exact start-codons
start_codons.sort_values("ortologus_row", inplace=True)
for row in tqdm(range(len(start_codons))):
    freqs = pd.Series(start_codons["start_codons"][row]).value_counts() # If it will be need to visualize percents, use normalize=True in brackets
    rowlength = len(start_codons["start_codons"][row])
    if "ATG" in freqs:
        start_codons["ATG"][row] = freqs["ATG"]
    else:
        start_codons["ATG"][row] = 0
    if "GTG" in freqs:
        start_codons["GTG"][row] = freqs["GTG"]
    else:
        start_codons["GTG"][row] = 0
    if "TTG" in freqs:
        start_codons["TTG"][row] = freqs["TTG"]
    else:
        start_codons["TTG"][row] = 0

  0%|                                                                                                               | 0/1525 [00:00<?, ?it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

  7%|███████▍                                                                                           | 114/1525 [00:00<00:01, 1137.87it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["TTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = freqs["GTG"]
/tmp/ipykernel_107740/1178007494.py:31: SettingWithCopyWarning: 
A value

/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["TTG"][row] = 0
 15%|██████████████▊                                                                                    | 228/1525 [00:00<00:01, 1052.40it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

 22%|█████████████████████▋                                                                             | 334/1525 [00:00<00:01, 1044.85it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

...

 79%|█████████████████████████████████████████████████████████████████████████████▌                    | 1206/1525 [00:01<00:00, 1019.91it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

 86%|████████████████████████████████████████████████████████████████████████████████████▍             | 1314/1525 [00:01<00:00, 1035.13it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["TTG"][row] = 0
 94%|████████████████████████████████████████████████████████████████████████████████████████████▎     | 1436/1525 [00:01<00:00, 1080.61it/s]/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_107740/1178007494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["TTG"][row] = 0
/tmp/ipykernel_107740/1178007494.py:23: SettingWithCopyWarning: 
A value is trying 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1525/1525 [00:01<00:00, 1044.52it/s]


In [15]:
# Computing uniformity of start-codon per ortologus row
start_codons["uniformity"] = "NA"
for row in range(len(start_codons)):
    if len(set(start_codons.iloc[row, 1])) == 1:
        start_codons.iloc[row, 5] = "uniform"
    else:
        start_codons.iloc[row, 5] = "non-uniform"

In [16]:
# Constructing table withh all data about the ortologus rows (including pararows)
start_codons2 = start_codons.merge(orto_rows, on="ortologus_row", how="outer")

In [17]:
# Creating a table, combining data about gene and its start-codone
strain_gene_row = start_codons2[["Strains", "Genes", "ortologus_row", "uniformity", "ATG", "GTG", "TTG"]]   # Other codons deleted
summary_rows = df1.merge(strain_gene_row, on="ortologus_row")

In [20]:
# Identifying non-uniform rows


,id,source,n_sequence,aa_sequence,type_of_DNA_source,type_of_the_gene,start_codone,product,cog,p_c_unity,additional_info,first,last,strand,length,ortologus_row
0,LEBECCED_00001,C_psittaci4_1,GTGGTAGACTTTGCAACTTTAGGTGGTAGACTTTTGGTTGAAAAAA...,MVDFATLGGRLLVEKNLDCARLVHYKILRLSMSGLSHHHKSRLFLT...,plasmid,NaN,GTG,hypothetical protein,absent,4,"('4', 1)",16,1033,-1,1017,978
1,LEBECCED_00002,C_psittaci4_1,GTGAGTAAATTAACTAAAGAAGCATCCGCTTTTTTTCAAAAGAATC...,MSKLTKEASAFFQKNQENTTKEFLKKEFAMDVFSVSLSDIEKEQIE...,plasmid,NaN,GTG,hypothetical protein,absent,4,"('4', 2)",1161,1902,-1,741,984
2,LEBECCED_00003,C_psittaci4_1,GTGAAAACGCTAGCATTTTGTTCTTTTAAAGGTGGTACTGGGAAAA...,MKTLAFCSFKGGTGKTTLSLNIGSNLAQVSRKKVLLVDLDPQANLT...,plasmid,NaN,GTG,Sporulation initiation inhibitor protein Soj,COG1192,4,"('4', 3)",1898,2678,-1,780,981
3,LEBECCED_00004,C_psittaci4_1,GTGGTTTTATATGAAAAAATCATACGTCTATTTACTGGAGAGATTA...,MVLYEKIIRLFTGEIMNDKQNMTNDFIKIVKDVEKDFPELDIKMKV...,plasmid,NaN,GTG,hypothetical protein,absent,4,"('4', 4)",2708,3062,-1,354,979
4,LEBECCED_00005,C_psittaci4_1,ATGGGTAATTCTGGTTTTTACTTAAATGACACCCAAAATTGTGTAT...,MGNSGFYLNDTQNCVFADNIKLGQMESPLQDQQLILGTKSTPTAAK...,plasmid,NaN,ATG,Virulence plasmid protein pGP3-D,absent,4,"('4', 5)",3111,3906,-1,795,974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27330,OCODBOIF_01040,C_psittaci11_1,ATGCTTACGGATAAAATTATATTATTTGTTACCGAAGACAGTAGTA...,MLTDKIILFVTEDSSISLQLKEFAQNVEYNIVVSSTLTDTPEADLI...,chromosome,NaN,ATG,Atypical response regulator protein ChxR,absent,11,"('11', 1040)",1158901,1159615,1,714,504
27331,OCODBOIF_01041,C_psittaci11_1,ATGATACTTGCATCTGTTTTATTTTCCCCCGAAGATTTTTCTTTTC...,MILASVLFSPEDFSFPELITEAYYTWDILALIDRKLSTHVFSGIHG...,chromosome,NaN,ATG,Bifunctional protein GlmU,absent,11,"('11', 1041)",1159674,1160289,1,615,448
27332,OCODBOIF_01042,C_psittaci11_1,ATGGTCGTTATGGATATTTTCGAAGCTTACCGCGTAATGATAGAAA...,MVVMDIFEAYRVMIETGIEDSLDEFGPKGFSIRAPVEYALTSGGKR...,chromosome,NaN,ATG,Farnesyl diphosphate synthase,absent,11,"('11', 1042)",1160304,1161186,1,882,706
27333,OCODBOIF_01044,C_psittaci11_1,ATGAAAAACCTTATCGATAATAACTTAGTGAGATTTAAAAATATTT...,MKNLIDNNLVRFKNISKTKQGIFVNFKVRGEKGGASFTASIAVDIE...,chromosome,NaN,ATG,hypothetical protein,absent,11,"('11', 1044)",1161285,1161540,-1,255,87


We need to align only rows with different start-codons.

In [9]:
non_uniform_or_list = list(set(start_codons2.query("uniformity == 'non-uniform'").ortologus_row))
for orto_row in tqdm(non_uniform_or_list):
    subset = summary_rows.query("ortologus_row == @orto_row")
    with open (f"../{folder_name}/data/multialignments/{folder_name}_withoutp_{str(orto_row)}.fasta", "w") as nucleotide_fasta:
        for index, row in subset.iterrows():
            if row['type_of_the_gene'] != "pseudogene":
                nucleotide_fasta.write(">")
                nucleotide_fasta.write(row["id"])
                nucleotide_fasta.write("_")
                nucleotide_fasta.write(row["source"])
                nucleotide_fasta.write("_")
                nucleotide_fasta.write(row["start_codone"])
                nucleotide_fasta.write("\n")
                nucleotide_fasta.write(row["n_sequence"])
                nucleotide_fasta.write("\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 402.69it/s]


#### Saving summary-rows and start-codons2

In [18]:
summary_rows.to_csv(f"../{folder_name}/data/summary_rows_prokka.csv", index=False)

In [19]:
start_codons2.to_csv(f"../{folder_name}/data/start_codons2_prokka.csv", index=False)